In [2]:
from datasets import load_dataset
import pandas as pd
import numpy as np
import requests
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, get_linear_schedule_with_warmup
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import AdamW
from torch.nn.utils import clip_grad_norm_
from tqdm.notebook import tqdm
from sklearn.metrics import classification_report, confusion_matrix

In [5]:
tokeniser = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [29]:
def encode(corpus):
    encoded = tokeniser.batch_encode_plus(corpus, max_length=128,
                                         add_special_tokens=True,
                                         return_attention_mask=True,
                                         truncation=True,
                                         return_tensors='pt',
                                         padding='max_length')

    return encoded['input_ids'], encoded['attention_mask']


def get_dataloader(ids, masks, data, isRandom):
    tensored = TensorDataset(ids, masks, data)
    sampler = RandomSampler(tensored) if isRandom else SequentialSampler(tensored)
    dataloader = DataLoader(tensored, sampler=sampler, batch_size=16)
    return dataloader

In [7]:
dataset = load_dataset('dair-ai/emotion', name='unsplit', split='train')

Found cached dataset emotion (/home/zum/.cache/huggingface/datasets/dair-ai___emotion/unsplit/1.0.0/cca5efe2dfeb58c1d098e0f9eeb200e9927d889b5a03c67097275dfb5fe463bd)


In [8]:
data_labels = dataset.features['label'].names
data_labels

['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']

In [9]:
dataset.set_format(type='pandas')

In [10]:
N = len(set(dataset['label']))
all_data = dataset[:]

In [11]:
N

6

In [15]:
# Smaller
all_data = all_data.groupby('label').apply(lambda x: x.sample(500)).reset_index(drop=True)

In [16]:
x_train, x_test, y_train, y_test = train_test_split(all_data['text'], all_data['label'], test_size=0.15, random_state=1)

In [18]:
print(x_train.shape, x_test.shape)

(2550,) (450,)


In [19]:
train_ids, train_masks = encode(list(x_train))
test_ids, test_masks = encode(list(x_test))

In [20]:
y_train = torch.LongTensor(list(y_train))
y_test = torch.LongTensor(list(y_test))

In [21]:
train_loader = get_dataloader(train_ids, train_masks, y_train, True)
test_loader = get_dataloader(test_ids, test_masks, y_test, False)

In [23]:
classifier = BertForSequenceClassification.from_pretrained('bert-base-uncased',
                                                          num_labels=N,
                                                          output_attentions=False,
                                                          output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [23]:
device='cpu'
classifier = classifier.cpu()

In [24]:
optimiser = AdamW(classifier.parameters(), lr=2e-6)
scheduler = get_linear_schedule_with_warmup(optimiser,
                                           num_training_steps=len(train_loader) * 5,
                                           num_warmup_steps=0)

In [26]:
def save_model(model, path):
    torch.save(model, path)

In [ ]:
train_losses = []

for epoch in range(5):
    classifier.train()
    train_loss = 0
    
    for step, data in enumerate(tqdm(train_loader, desc='Training')):
        ids, masks, labels = [x.to(device) for x in data]
        output = classifier(input_ids=ids, attention_mask=masks, labels=labels)
        loss = output.loss
        train_loss += loss.item()
        
        classifier.zero_grad()
        loss.backward()
        del loss
        
        clip_grad_norm_(parameters=classifier.parameters(), max_norm=1.0)
        optimiser.step()
        scheduler.step()
    
    if epoch % 5 == 0:
        save_model(classifier, './secondtry' + str(epoch))
    train_losses.append(train_loss / (step + 1))
    print("Loss: {}".format(train_loss / (step + 1)))
      

Training:   0%|          | 0/144 [00:00<?, ?it/s]

In [19]:
save_model(classifier, './finalsecondtry')

In [20]:
classifier.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [3]:
classifier = torch.load('./finalsmall')

In [4]:
torch.save(classifier.state_dict(), './finalsmalldict')

In [12]:
classifier = BertForSequenceClassification.from_pretrained('bert-base-uncased',
                                                          num_labels=6,
                                                          output_attentions=False,
                                                          output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [13]:
classifier.load_state_dict(torch.load('./finalsmalldict'))

<All keys matched successfully>

In [25]:
test_pred = []
test_loss = 0

with torch.no_grad():
    for data in tqdm(test_loader):
        ids, mask, labels = [x.to(device) for x in data]
        output = classifier(input_ids=ids, attention_mask=mask, labels=labels)

        loss = output.loss
        test_loss += loss.item()

        test_pred.append(np.argmax(output.logits.cpu().detach().numpy(),axis=-1))
test_pred = np.concatenate(test_pred)

  0%|          | 0/29 [00:00<?, ?it/s]

In [28]:
print(classification_report(test_pred, list(y_test),target_names=data_labels))

              precision    recall  f1-score   support

     sadness       0.95      0.92      0.93        76
         joy       0.81      0.98      0.89        58
        love       0.97      0.86      0.91        70
       anger       0.87      0.96      0.91        68
        fear       0.92      0.87      0.89        91
    surprise       0.94      0.90      0.92        87

    accuracy                           0.91       450
   macro avg       0.91      0.91      0.91       450
weighted avg       0.91      0.91      0.91       450

